# python profiling

- 파이썬은 동적 특성 때문에 느릴 거라고 생각한 함수가 굉장히 빠를 수도 있고 당연히 빠를 거라고 생각한 동작들이 속도가 느릴 수 있음
- 가장 좋은 방법은 최적화하기 전에 직관을 무시하고 직접 프로그램의 성능을 측정해보는 것
- 파이썬은 프로그램의 어느 부분이 실행 시간을 소비하는지 파악할 수 있도록 내장 프로파일러(profiler)를 제공
- 프로파일러를 이용하면 문제의 가장 큰 원인에 최적화 노력을 최대한 집중할 수 있고, 속도에 영향을 주지 않는 부분은 무시할 수 있음

## profiler 종류
- profile: 순수 파이썬 내장 프로파일러(잘 사용x)
- cProfile: C 확장 모듈 프로파일러
- timeit: 시간 체크만 간단하게 하고 싶을 때 사용
- line_profiler: line by line으로 성능 체크 가능

- 예를 들어 프로그램의 알고리즘이 느린 이유를 알고 싶다고 가정. 다음은 삽입 정렬로 데이터 리스트를 정렬하는 함수

In [29]:
def insertion_sort(data):
    result = []
    for value in data: 
        insert_value(result, value) 
    return result

# 삽입 정렬의 핵심메커니즘은 각 데이터의 삽입 지점을 찾는 함수
# 아래는 극히 비효율적인 insert_value 함수로 입력 배열을 순차적으로 스캔 

def insert_value(array, value): 
    for i, existing in enumerate(array): 
        if existing > value: 
            array.insert(i, value) 
            return 
    array.append(value) 

# insertion_sort와 insert_value를 프로파일하려고 난수로 구성된 데이터 집합을 생성하고, 프로파일러에 넘길 test 함수를 정의 

from random import randint 

max_size = 10 ** 4 
data = [randint(0, max_size) for _ in range(max_size)] 
test = lambda: insertion_sort(data)

# profile & cProfile
- 보통 cProfile 사용. 순수 내장 프로파일러는 성능 체크 결과를 왜곡시킬 만큼 부하가 많다고 함
- 테스트 함수의 실행이 끝나면 내장 모듈 pstats와 Stats 클래스로 함수의 성능 통계를 뽑을 수 있음.
- Stats 객체의 다양한 메서드를 이용하면 프로파일 정보를 선택하고 정렬하는 방법을 조절해서 관심 있는 정보만 볼 수 있음

In [18]:
import cProfile
def f(x):
    for _ in range(1000):
        x += 1
    return x
cProfile.run('f(12)')

# ????????? 가장 간단한 활용 사례라고 하는데 시간이 항상 0으로 나옴

         4 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-18-228deb4ab093>:2(f)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [32]:
from cProfile import Profile
from pstats import Stats 

profiler = Profile()
profiler.runcall(test)

stats = Stats(profiler)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_stats()

         20003 function calls in 1.653 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.653    1.653 <ipython-input-31-2a588cbf71de>:23(<lambda>)
        1    0.006    0.006    1.653    1.653 <ipython-input-31-2a588cbf71de>:1(insertion_sort)
    10000    1.630    0.000    1.648    0.000 <ipython-input-31-2a588cbf71de>:10(insert_value)
     9991    0.018    0.000    0.018    0.000 {method 'insert' of 'list' objects}
        9    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




- 결과는 함수로 구성된 정보의 테이블. 데이터 샘플은 runcall 메서드가 실행되는 동안 프로파일러가 활성화되어 있을 때만 얻어옴
- 프로파일러 통계 컬럼의 의미
    - ncalls: 프로파일링 주기 동안 함수 호출 횟수
    - tottime: 함수가 실행되는 동안 소비한 초 단위의 시간으로, 다른 함수 호출을 실행하는 데 걸린 시간은 배제
    - percall : sub function을 제외한 1회 실행시간
    - tottime percall: 함수를 호출하는 데 걸린 평균 시간이며 초 단위. 다른 함수의 호출 시간은 배제. tottime을 ncalls로 나눈 값
    - cumtime: 함수를 실행하는 데 걸린 초 단위 누적 시간이며, 다른 함수를 호출하는 데 걸린 시간도 포함
    - cutime percall: 함수를 호출할 때마다 걸린 시간에 대한 초 단위 편균 시간이며, 다른 함수를 호출하는 데 걸린 시간도 포함. cumtime을 ncalls로 나눈 값
- 더 자세한 설명: [공식 홈페이지](https://docs.python.org/2/library/profile.html])

- 앞의 프로파일러 통계 테이블을 보면 테스트에서 CPU를 가장 많이 사용하는 부분은 insert_value 함수에서 소비한 누적 시간
- 이번에는 내장 모듈 bisect를 사용하도록 insert_value 함수를 재정의합

In [4]:
from bisect import bisect_left

def insert_value(array, value): 
    i = bisect_left(array, value) # 수정
    array.insert(i, value)
    
    
profiler = Profile()
profiler.runcall(test)

stats = Stats(profiler)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_stats()

         30003 function calls in 0.048 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.048    0.048 <ipython-input-1-5af51e4b3ebc>:23(<lambda>)
        1    0.009    0.009    0.048    0.048 <ipython-input-1-5af51e4b3ebc>:1(insertion_sort)
    10000    0.015    0.000    0.039    0.000 <ipython-input-4-2a62c4d454b7>:3(insert_value)
    10000    0.015    0.000    0.015    0.000 {method 'insert' of 'list' objects}
    10000    0.008    0.000    0.008    0.000 {built-in method _bisect.bisect_left}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




- 프로파일러를 실행하여 새 프로파일러 통계를 생성한 결과, 새로운 함수는 더 빨라졌고 누적 시간은 이전의 insert_value 함수에 비해 거의 100배 이상 단축
- 때로는 전체 프로그램을 프로파일할 때 공통 유틸리티 함수에서 대부분의 실행 시간을 소비할 수도 있음
- 프로파일러의 기본 출력은 유틸리티 함수가 프로그램의 다른 부분에서 얼마나 많이 호출되는지 보여주지 않기 때문에 이해하기 어려움
- 예를 들어 다음 my_utility 함수는 프로그램에 있는 다른 두 함수에서 반복적으로 호출

In [5]:
from cProfile import Profile 
from pstats import Stats 

def my_utility(a, b): 
    pass 

def first_func(): 
    for _ in range(1000): 
        my_utility(4, 5) 

def second_func(): 
    for _ in range(10): 
        my_utility(1, 3) 

def my_program(): 
    for _ in range(20): 
        first_func() 
        second_func() 

test = lambda: my_program() 

profiler = Profile() 
profiler.runcall(test) 

stats = Stats(profiler) 
stats.strip_dirs() 
stats.sort_stats('cumulative') 
stats.print_stats()

         20243 function calls in 0.018 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.018    0.018 <ipython-input-5-ff5707c47a90>:20(<lambda>)
        1    0.000    0.000    0.018    0.018 <ipython-input-5-ff5707c47a90>:15(my_program)
       20    0.010    0.000    0.017    0.001 <ipython-input-5-ff5707c47a90>:7(first_func)
    20200    0.008    0.000    0.008    0.000 <ipython-input-5-ff5707c47a90>:4(my_utility)
       20    0.000    0.000    0.000    0.000 <ipython-input-5-ff5707c47a90>:11(second_func)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




- my_utility 함수가 대부분의 실행 시간을 소비하는 원인이라는 게 명확하지만 이 함수가 이렇게 많이 호출되는 이유는 명확하게 알기 어려움
- 프로그램 코드에서 찾는다면 my_utility가 여러 번 호출되고 있다는 사실을 알겠지만 여전히 이해하기 쉽지 않음
- 파이썬 프로파일러는 이 문제를 처리하려고 각 함수의 프로파일링 정보에 기여하는 호출자를 찾는 방법을 제공

In [6]:
stats.print_callers()

   Ordered by: cumulative time

Function                                          was called by...
                                                      ncalls  tottime  cumtime
<ipython-input-5-ff5707c47a90>:20(<lambda>)       <- 
<ipython-input-5-ff5707c47a90>:15(my_program)     <-       1    0.000    0.018  <ipython-input-5-ff5707c47a90>:20(<lambda>)
<ipython-input-5-ff5707c47a90>:7(first_func)      <-      20    0.010    0.017  <ipython-input-5-ff5707c47a90>:15(my_program)
<ipython-input-5-ff5707c47a90>:4(my_utility)      <-   20000    0.008    0.008  <ipython-input-5-ff5707c47a90>:7(first_func)
                                                         200    0.000    0.000  <ipython-input-5-ff5707c47a90>:11(second_func)
<ipython-input-5-ff5707c47a90>:11(second_func)    <-      20    0.000    0.000  <ipython-input-5-ff5707c47a90>:15(my_program)
{method 'disable' of '_lsprof.Profiler' objects}  <- 




- 위 프로파일러 통계 테이블은 호출된 함수를 왼쪽에 보여주며, 누가 이런 호출을 하는지를 오른쪽에 보여줌
- 결과적으로  my_utility가 first_func에서 가장 많이 사용되었음을 명확하게 확인 가능

- cProfile이 더 정확한 프로파일링 정보를 제공하므로 profile 모듈 대신에 cProfile을 사용
- Profile 객체의 runcall 메서드는 함수 호출 트리를 프로파일하는 데 필요한 모든 기능을 제공
- Stats 객체는 프로그램 성능을 이해하는 데 필요한 프로파일링 정보를 선택하고 출력하는 기능을 제공

# line_profiler
- 각 라인별로 프로파일링 하고 싶은 경우는 line_profiler와 flame graph가 많이 사용됨
- 가장 쉬운 방법은 `kernprof` 스크립트를 사용하는 것
- 프로파일링 할 함수를 수동으로 지정하고 실행 시 패키지를 호출하는 방식

```bash
pip3 install line_profiler
kernprof -l script_to_profile.py
```
- `kernprof`는 `LineProfiler`의 인스턴스를 생성하고 이름 프로필을 사용하여 `__builtins__` 네임 스페이스에 삽입
- 데코레이터로 사용하도록 작성되었으므로 스크립트에서 `@profile`로 프로파일링하려는 함수를 꾸며 줌

In [ ]:
# 라이브러리 import 해줄 필요 없이 프로파일링 하고 싶은 function에 데코레이터만 붙여 주면 됨
# 그냥 바로 실행은 되지 않고 bash에서 "kernprof -l script_to_profile.py" 형태로 실행

@profile
def do_stuff(numbers):
    print(numbers)

numbers = 2
do_stuff(numbers)

- 아래는 실행 결과
- 결과물로 `profiler_test.py.lprof`라는 파일이 생성되는데, 바이너리 파일이므로 그냥 읽을 수는 없음
```bash
(data) lynn@ip-172-31-6-44:~/workspace/interpy-kr/day4$ kernprof -l profiler_test.py 
2
Wrote profile results to profiler_test.py.lprof
```

- 아래 명령어로 `README` 형태로 취급하여 읽어 냄

```bash
(data) lynn@ip-172-31-6-44:~/workspace/interpy-kr/day4$ python -m line_profiler profiler_test.py.lprof 
Timer unit: 1e-06 s

Total time: 0.000165 s
File: profiler_test.py
Function: do_stuff at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           @profile
     2                                           def do_stuff(numbers):
     3         1        165.0    165.0    100.0      print(numbers)
```

- `-v` 옵션을 붙이면 실행 즉시 결과를 볼 수 있음

```bash
(data) lynn@ip-172-31-6-44:~/workspace/interpy-kr/day4$ kernprof -v -l profiler_test2.py 
2
Wrote profile results to profiler_test2.py.lprof
Timer unit: 1e-06 s

Total time: 0.000151 s
File: profiler_test2.py
Function: do_stuff at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           @profile
     2                                           def do_stuff(numbers):
     3         1        151.0    151.0    100.0      print(numbers)

```

## (참고) pprofile
- 항상 line_profiler는 글로벌하게 깔린 python을 바라보도록 되어 있어 특정 가상환경 내의 인터프리터를 바라보도록 할 수 없음
- `pprofile`은 virtualenv에서 line_profile을 쓰기 힘들어 만들어 놓은 라이브러리
- 공식 사이트: [주소 참고](https://pypi.python.org/pypi/pprofile/1.6.2)

- 설치
    - `virtualenv -p pypy경로 venv` (pypy 설치를 위해서 -p 옵션)
    - `cd venv`
    - `source bin/activate`
    - `pip install pprofile`
- 사용법
    - `pprofile 파일명`

# timeit
- 명령 행과 파이썬 쉘에서 하나의 라이너 또는 작은 표현식을 테스트하는 간단한 방법을 제공
- 벤치마킹용으로 간단하게 사용 시에는 가장 유용

In [28]:
# 스크립트에서 바로 실행 시
import timeit
timeit.timeit('"-".join(str(n) for n in range(100))', number=10000)

0.2576766051352024

In [33]:
from timeit import timeit

def mkList1():
    result = []
    for value in range(1000):
        result.append(value)
    return result


def mkList2():
    result = [value for value in range(1000)]
    return result


print("mkList1 : ", timeit(mkList1, number=1000), "s")
print("mkList2 : ", timeit(mkList2, number=1000), "s")

mkList1 :  0.08979501202702522 s
mkList2 :  0.0355030819773674 s


command 창에서 실행 시
```bash
(data) lynn@ip-172-31-6-44:~/workspace/interpy-kr/day4$ python -m timeit '"-".join(str(n) for n in range(100))'
10000 loops, best of 3: 24 usec per loop
```

# Wrap-up
- 성능 저하를 일으키는 원인이 때로는 불분명하므로 파이썬 프로그램을 최적화하기 전에 프로파일을 해야 함
- 스크립트의 병목 현상을 발견 할 수 있어야 함
- 그러나 최적화가 데이터 구조의 부실한 선택이나 알고리즘 설계의 결함을 보완 할 수는 없다는 점에 유의
- 두 번째로 가독성 / 디자인 / 품질을 희생하면서 코딩 프로세스를 너무 일찍 최적화하려고하지 말 것
- pycharm 같은 IDE에서도 프로파일링 기능 제공함: [참고](https://www.jetbrains.com/help/pycharm/profiler.html)

# Reference

- https://brownbears.tistory.com/248 [불곰]
- https://riptutorial.com/ko/python/example/20706/%EC%BD%94%EB%93%9C-%ED%94%84%EB%A1%9C%ED%8C%8C%EC%9D%BC-%EB%A7%81
- https://hashcode.co.kr/questions/2311/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%BD%94%EB%93%9C%EB%A5%BC-%ED%94%84%EB%A1%9C%ED%8C%8C%EC%9D%BC%EB%A7%81%ED%95%98%EA%B8%B0
- https://umbum.tistory.com/789
- http://free-lunch.github.io/python-profiler/